This script analyzes and compares different mathematical approximations for gyroaveraging operators used in plasma physics, specifically in the context of gyrokinetics. The primary goal is to visualize the accuracy of Padé approximants against their "exact" counterparts, which involve exponential and Bessel functions.

The script defines several key functions, we write them here using the species-wise normalized squared wave number:
$$ b_s^2 = k_\perp^2 \rho_s^2 $$

The linearized gyrokinetic quasi-neutrality equation writes:
$$ \sum_s \frac{n_{0s} q_s^2}{T_{0s}} [1 - \Gamma_0^s(b_s^2)] \phi = \sum_s q_s \int d^3v \, \Gamma_1^s(b_s^2) F_s $$
In the following we'll refer to $ P=\sum_s \frac{n_{0s} q_s^2}{T_{0s}} [1 - \Gamma_0^s(b_s^2)] $ as the polarization operator, and $\sigma = \sum_s q_s \int d^3v \, \Gamma_1^s(b_s^2) F_s $ as the gyroaveraged charge density.
We will evaluate it for different approximations and species combinations.

**Exact Gyroaveraging Operators**:
- The $\Gamma_1^s$ operator is defined as:
    $$ \Gamma_{1}^s(b_s^2) = \exp[-b_s^2/2\big] $$
- The $\Gamma_0^s$ operator is defined using the modified Bessel function of the first kind, $I_0$:
    $$ \Gamma_{0}^s(b_s^2) = \Gamma_{1}^s(b_s^2) \cdot I_0[b_s^2] $$

**Padé Approximants for Gyroaveraging Operators**:
- The (1,4) order Padé approximant for $\Gamma_1^s$ is (Held et al. 2020, Eq. 37):
    $$ \Gamma_{1,Padé}^s(b_s^2) = \frac{1}{1 + b_s^2/2} $$
- The Padé approximant for $\Gamma_0^s$ is obtain by considering $\Gamma_1 \approx \sqrt{\Gamma_0}$, yielding:
    $$ \Gamma_{0,Padé}^s(b_s^2) = \frac{1}{1 + b_s^2} $$


## Impact of the polarization error on the obtained potential
Here we evaluate the error in the potential obtained through an approximation of the polarization operator in the linearized gyrokinetic quasi-neutrality equation. We right in a compact form the quasi-neutrality equation as:
$$ P\phi = \sigma $$
where $P = \sum_s \frac{n_{0s} q_s^2}{T_{0s}} [1 - \Gamma_0^s(b_s^2)]$ is the polarization operator and $ \sigma = \sum_s q_s \int d^3v \, \Gamma_1^s(b_s^2) F_s $ is the gyroaveraged charge density.

### Gauss error propagation
Through standard error propagation, we can write the error in the potential as:
$$ \delta \phi = \frac{\partial \phi}{\partial P} \delta P + \frac{\partial \phi}{\partial \sigma} \delta \sigma $$
which gives:
$$ \delta \phi = -\frac{\sigma}{P^2} \delta P + \frac{1}{P} \delta \sigma $$
We can then evaluate the relative error on $\phi$ as:
$$ \frac{\delta \phi}{\phi} = -\frac{\delta P}{P} + \frac{\delta \sigma}{\sigma} $$
The error on the charge density can be evaluated through the error in $\Gamma_1$ as:
$$ \delta \sigma = \sum_s \frac{\partial \sigma}{\partial \Gamma_1} \delta \Gamma_1^s = \sum_s q_s n_{0s} \delta \Gamma_1^s $$
Where we considered that $\int d^3v F_s = n_{0s}$.
The relative error on $\phi$ becomes:
$$ \frac{\delta \phi}{\phi} = -\frac{\delta P}{P} + \frac{\sum_s q_s n_{0s} \delta \Gamma_1^s}{\sigma_0} $$
where $\sigma_0 = \sum_s q_s n_{0s} \Gamma_{1}^s$ is the exact reference charge density.

### Simplification of the multi-ion species case
It is not trivial to invert the polarization operator $P$ when different ions are present. 
We introduce the single ion and average ion approximations to simplify the analysis.
- **Single ion approximation**: we assume that all ion species are similar to a given ion $j$. All species have charge $q_j$, density $n_0=\sum_i n_{i0}$ and temperature $T_{j0}$. The polarization operator becomes:
$$ P_{single} = \frac{n q_j^2}{T_{0j}} [1 - \Gamma_0^j] $$
- **Average ion approximation**: we consider an average ion species with charge $\bar q = \sum_i q_i n_{i0}/n_{0}$, density $n_{0}$ and temperature $\bar T = \sum_i n_{i0} T_{i0} /n_{0}$
$$ P_{avg} = \frac{n_0 \bar q^2}{\bar T} [1 - \bar \Gamma_0] $$
where $\bar \Gamma_0$ is evaluated at the average ion Larmor radius $\bar \rho^2 = \sum_i n_{i0} \rho_i^2 / n_0$.

In [ ]:
import pygkyl
import numpy as np
import matplotlib.pyplot as plt
import src.flr_operators_inc as flr

# Define the different ion species
Dplus   = pygkyl.Species(name = 'D+',   q = 1.0*flr.eV,  m = 2.014*flr.amu,     T0 = 100*flr.eV, n0 = 1e19)
Tplus   = pygkyl.Species(name = 'T+',   q = 1.0*flr.eV,  m = 3.016*flr.amu,     T0 = 100*flr.eV, n0 = 1e19)
W10plus = pygkyl.Species(name = 'W10+', q = 10*flr.eV,   m = 183.84*flr.amu,    T0 = 100*flr.eV, n0 = 5e16)
W45plus = pygkyl.Species(name = 'W45+', q = 45*flr.eV,   m = 183.84*flr.amu,    T0 = 100*flr.eV, n0 = 5e16)
Wnplus  = pygkyl.Species(name = 'Wn+',  q = 45*flr.eV,   m = 183.84*flr.amu,    T0 = 100*flr.eV, n0 = 5e16) # variable Tungsten species for scans
Wnplus.color = 'purple'

In [ ]:
ctx = flr.Context(ion_list=[Dplus, Tplus, W10plus, W45plus])
# We can change the magnetic field strength to verify that the results are independent of it
Bref = 2.0 # Tesla
ctx.set(Bref=Bref)

nbar = np.sum([s.n0 for s in ctx.species])
qbar = np.sum([s.q * s.n0 for s in ctx.species]) / nbar
mbar = np.sum([s.m * s.n0 for s in ctx.species]) / nbar
Tbar = np.sum([s.T0 * s.n0 for s in ctx.species]) / nbar
avg_ion = pygkyl.Species(name = 'avg ion', q = qbar, n0 = nbar, m = mbar, T0 = Tbar, Bref=ctx.Bref)

single_ion = pygkyl.Species(name = 'D+', q = Dplus.q, n0 = ctx.qtot_ion/Dplus.q, m = Dplus.m, T0 = Dplus.T0, Bref=ctx.Bref)

rhogrid = ctx.rhogrid
kgrid = ctx.kgrid
KK, RR = np.meshgrid(kgrid, rhogrid, indexing='ij')

fig, ax_grid = plt.subplots(3,2, figsize=(8,10))
ax = ax_grid.flatten()

x = kgrid * Dplus.rho
## First axis for gamma0
for i, s in enumerate(ctx.species):
    color = s.color
    ax[0].plot(x, flr.pol_species(kgrid, s, ctx.Bref, G0func=flr.gamma0_ex), color=color)
    ax[0].plot(x, flr.pol_species(kgrid, s, ctx.Bref, G0func=flr.gamma0_pade), color=color, linestyle='--')

# add a text box near the line
# ax[0].axvline(x=ctx.kmax * Dplus.rho, color='k', linestyle='-', alpha=0.2)
# ymax = ax[0].get_ylim()[1]
# ax[0].text(ctx.kmax * Dplus.rho*0.85, 0.8*ymax, r'96x96', rotation=90, verticalalignment='center', color='k', alpha=0.5)
ax[0].set_xlabel(r'$k_\perp \rho_D$')
# ax[0].set_ylabel(r'$\Gamma_0^s$')
ax[0].set_ylabel(r'$q_s^2 n_s / T_s (1 - \Gamma_0^s)$')
ax[0].set_title(r's-wise polarization operators')

## Second axis Gamma1
for i, s in enumerate(ctx.species):
    color = s.color
    ax[1].plot(x, flr.gamma1_ex(kgrid, s.rho), color=color)
    ax[1].plot(x, flr.gamma1_pade(kgrid, s.rho), color=color, linestyle='--')

ax[1].set_xlabel(r'$k_\perp \rho_D$')
ax[1].set_ylabel(r'$\Gamma_1^s$')
ax[1].set_title(r'Gyroaveraging Operators')
# ax[1].axvline(x=ctx.kmax * Dplus.rho, color='k', linestyle='-', alpha=0.2)

## Third axis
idx = 2
# Plot the error between of Gamma0
Zex = flr.gamma0_ex(KK, RR)
Zpade = flr.gamma0_pade(KK, RR)
Zerr = (Zex - Zpade)/Zex*100
im = ax[idx].pcolormesh(KK*Dplus.rho, RR/Dplus.rho, Zerr, shading='auto', vmin=-100, vmax=100)
im.cmap = 'bwr'
cbar = fig.colorbar(im, ax=ax[idx])
ax[idx].set_xlabel(r'$k_\perp \rho_D$')
ax[idx].set_ylabel(r'$\rho/\rho_D$')
ax[idx].set_title(r'$(\Gamma_0 - \Gamma_0^{pade})/\Gamma_0$ [\%]')
# plot a line at rho for each species
for i, s in enumerate(ctx.species):
    color = s.color
    # ax[idx].axhline(y=s.rho/Dplus.rho, linestyle='-', color=color, alpha=0.75)
    # ax[idx].axvline(x=Dplus.rho/s.rho, linestyle='-', color=color, alpha=0.75)
    ax[idx].scatter([Dplus.rho/s.rho], [s.rho/Dplus.rho], color=color, marker='o', s=100, alpha=0.75)
# ax[idx].axvline(x=ctx.kmax * Dplus.rho, color='k', linestyle='-', alpha=0.2)


## Fourth axis
idx = 3
# Plot the error between of Gamma1
Zex = flr.gamma1_ex(KK, RR)
Zpade = flr.gamma1_pade(KK, RR)
Zerr = (Zex - Zpade)/Zex*100
im = ax[idx].pcolormesh(KK*Dplus.rho, RR/Dplus.rho, Zerr, shading='auto', vmin=-100.0, vmax=100.0)
im.cmap = 'bwr'
cbar = fig.colorbar(im, ax=ax[idx])
ax[idx].set_xlabel(r'$k_\perp \rho_D$')
ax[idx].set_ylabel(r'$\rho/\rho_D$')
ax[idx].set_title(r'$(\Gamma_1 - \Gamma_1^{pade})/\Gamma_1$ [\%]')
# plot a line at rho for each species
for i, s in enumerate(ctx.species):
    color = s.color
    ax[idx].axhline(y=s.rho/Dplus.rho, linestyle='-', color=color, alpha=0.75)
    ax[idx].axvline(x=Dplus.rho/s.rho, linestyle='-', color=color, alpha=0.75)
    ax[idx].scatter([Dplus.rho/s.rho], [s.rho/Dplus.rho], color=color, marker='o', s=100, alpha=0.75)
# ax[idx].axvline(x=ctx.kmax * Dplus.rho, color='k', linestyle='-', alpha=0.2)

## Fifth axis
idx = 4
ax[idx].axis('off')
## Sixth axis
idx = 5

from matplotlib.lines import Line2D
legend_elements_species = [Line2D([0], [0], color=s.color, lw=1, label=f'{s.name} (n={s.n0:.1e}, \
    T={s.T0/flr.eV:.0f}eV)') for i, s in enumerate(ctx.species)]
legend_elements_style = [Line2D([0], [0], color='k', linestyle='-', lw=2, label='Exact'),
                         Line2D([0], [0], color='k', linestyle='--', lw=2, label='Padé')]
custom_legend = legend_elements_species + legend_elements_style
ax[idx].axis('off')
ax[idx].legend(handles=custom_legend, loc='center', ncol=1)
plt.tight_layout()

## Evaluate the relative error on the potential following the error propagation analysis
sigma_ex = ctx.get_charge_density(kgrid, flr.gamma1_ex)
pol_op_ex = ctx.polarization_op(kgrid, flr.gamma0_ex)
phi_ex = sigma_ex / pol_op_ex

sigma_pade = ctx.get_charge_density(kgrid, flr.gamma1_pade)
delta_sigma = sigma_pade - sigma_ex
pol_op_pade = ctx.polarization_op(kgrid, flr.gamma0_pade)
pol_op_single_ion = ctx.polarization_op_single_ion(kgrid, single_ion.n0, single_ion.q, single_ion.m, single_ion.T0, flr.gamma0_pade)
pol_op_avg_ion = flr.pol_species(kgrid, avg_ion, ctx.Bref, G0func=flr.gamma0_pade)

fig, axs = plt.subplots(2,2, figsize=(8,6))
axs = axs.flatten()

ax = axs[0]
ax.plot(x, sigma_ex, color='k', label='Exact')
ax.plot(x, sigma_pade, color='k', linestyle='--', label='Padé')
ax.set_ylabel(r'$\sigma$ [C/m$^3$]')
ax.set_xlabel(r'$k_\perp \rho_D$')
# ax.axvline(x=ctx.kmax * Dplus.rho, color='k', linestyle='-', alpha=0.2)
ax.legend()

ax= axs[1]
ax.plot(x, pol_op_ex, color='k', label='Exact')
ax.plot(x, pol_op_pade, color='k', linestyle='--', label='Padé all species')
ax.plot(x, pol_op_single_ion, color=single_ion.color, linestyle='-.', label='single ion (%s)'%single_ion.name)  
ax.plot(x, pol_op_avg_ion, color='gray', linestyle=':', label='avg ion')
ax.set_ylabel(r'P [C/V/m$^3$]')
ax.set_xlabel(r'$k_\perp \rho_D$')
# ax.axvline(x=ctx.kmax * Dplus.rho, color='k', linestyle='-', alpha=0.2)
ax.legend()

ax = axs[2]
ax.plot(x, phi_ex, color='k', label='Exact')
ax.plot(x, sigma_pade/pol_op_pade, color='k', linestyle='--', label='Padé all species')
ax.plot(x, sigma_pade/pol_op_single_ion, color=single_ion.color, linestyle='-.', label='single ion (%s)'%single_ion.name)
ax.plot(x, sigma_pade/pol_op_avg_ion, color='gray', linestyle=':', label='avg ion')
ax.set_ylabel(r'$\phi$ [V]')
ax.set_xlabel(r'$k_\perp \rho_D$')
# ax.axvline(x=ctx.kmax * Dplus.rho, color='k', linestyle='-', alpha=0.2)
ax.legend()

ax = axs[3]
for (pol_approx, label) in zip([pol_op_pade, pol_op_avg_ion, pol_op_single_ion], 
                               ['Padé all species', 'avg ion', 'single ion (%s)'%single_ion.name]):
    delta_pol_op = pol_approx - pol_op_ex
    phi_err = (delta_sigma - phi_ex * delta_pol_op) / (pol_op_ex) / phi_ex
    ax.plot(x, phi_err*100, label=label)
ax.set_ylabel(r'Relative error on $\phi$ [\%]')
ax.set_xlabel(r'$k_\perp \rho_D$')
ax.grid()
ax.legend()
plt.tight_layout()

In [ ]:
single_ion = Dplus
Bref = 2.0
ctx = flr.Context(ion_list=[Dplus, Wnplus], Bref=2.0)

kperprhoD = 2.5

kperp = kperprhoD / Dplus.rho

# Scan over the temperature and density ratios
nratios = np.logspace(-6, -2, 48)
Tratios = np.linspace(0.1, 2.0, 48)
pol_op_ex = np.zeros((len(nratios), len(Tratios)))
pol_op_pade = np.zeros((len(nratios), len(Tratios)))
pol_op_single_ion = np.zeros((len(nratios), len(Tratios)))
pol_op_avg_ion = np.zeros((len(nratios), len(Tratios)))

for i, nratio in enumerate(nratios):
    for j, Tratio in enumerate(Tratios):
        ctx.species_set('Wn+', n0=Dplus.n0*nratio, T0=Dplus.T0*Tratio)
        pol_op_ex[i,j] = ctx.polarization_op(kperp, flr.gamma0_ex)
        pol_op_pade[i,j] = ctx.polarization_op(kperp, flr.gamma0_pade)
        pol_op_single_ion[i,j] = ctx.polarization_op_single_ion(kperp, ctx.ntot_ion, single_ion.q, single_ion.m, single_ion.T0, flr.gamma0_pade)
        pol_op_avg_ion[i,j] = ctx.polarization_op_single_ion(kperp, ctx.ntot_ion, ctx.qavg_ion, ctx.mavg_ion, ctx.Tavg_ion,flr.gamma0_pade)

fig2, axs = plt.subplots(3,2, figsize=(8,9))
axs = axs.flatten()

ax = axs[0]
Z = (pol_op_ex - pol_op_pade)/np.abs(pol_op_ex)*100
im = ax.pcolormesh(Tratios, nratios, Z, shading='auto',cmap='bwr', vmin=-100, vmax=100)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'$T_{Wn+}/T_D$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'Exact - Pade ($k_\perp \rho_D=%.1f$)'%kperprhoD)

ax = axs[1]
Z = (pol_op_ex - pol_op_single_ion)/np.abs(pol_op_ex)*100
im = ax.pcolormesh(Tratios, nratios, Z, shading='auto',cmap='bwr', vmin=-100, vmax=100)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'$T_{Wn+}/T_D$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'Exact - Single Ion (%s) ($k_\perp \rho_D=%.1f$)'%(single_ion.name, kperprhoD))

ax = axs[2]
Z = (pol_op_ex - pol_op_pade)/(2*pol_op_ex - pol_op_pade)*100
im = ax.pcolormesh(Tratios, nratios, Z, shading='auto',cmap='bwr', vmin=-100, vmax=100)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'$T_{Wn+}/T_D$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'$\delta\phi/\phi$ ($k_\perp \rho_D=%.1f$)'%kperprhoD)

ax = axs[3]
Z = (pol_op_ex - pol_op_avg_ion)/np.abs(pol_op_ex)*100
im = ax.pcolormesh(Tratios, nratios, Z, shading='auto',cmap='bwr', vmin=-100, vmax=100)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'$T_{Wn+}/T_D$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'Exact - Avg Ion ($k_\perp \rho_D=%.1f$)'%kperprhoD)

plt.tight_layout()
plt.show()

In [ ]:
# Define the different species
Dplus   = pygkyl.Species(name = 'D+',   q = 1.0*flr.eV, m = 2.014*flr.amu, T0 = 100*flr.eV, n0 = 1e19)
Wnplus = pygkyl.Species(name = 'Wn+', q = 45*flr.eV, m = 183.84*flr.amu, T0 = 100*flr.eV, n0 = 1e15)
Wnplus.color = 'purple'
single_ion = Dplus

Bref = 2.0
ctx = flr.Context(ion_list=[Dplus, Wnplus], Bref=2.0)

kperprhoD = 1.5

kperp = kperprhoD / Dplus.rho

# Colorbar limits
vmin_pade, vmax_pade = -100, 100
vmin_single, vmax_single = -100, 100
vmin_avg, vmax_avg = -100, 100
vmin_phi, vmax_phi = -100, 100


# Scan over the charge number and density ratios
nratios = np.logspace(-6, -2, 48)
qratios = np.linspace(1, 50, 50)
pol_op_ex = np.zeros((len(nratios), len(qratios)))
pol_op_pade = np.zeros((len(nratios), len(qratios)))
pol_op_pade_single_ion = np.zeros((len(nratios), len(qratios)))
pol_op_pade_avg_ion = np.zeros((len(nratios), len(qratios)))

for i, nratio in enumerate(nratios):
    for j, qratio in enumerate(qratios):
        ctx.species_set('Wn+', n0=Dplus.n0*nratio, q=qratio*flr.eV)
        pol_op_ex[i,j] = ctx.polarization_op(kperp, flr.gamma0_ex)
        pol_op_pade[i,j] = ctx.polarization_op(kperp, flr.gamma0_pade)
        pol_op_pade_single_ion[i,j] = ctx.polarization_op_single_ion(kperp, ctx.ntot_ion, single_ion.q, single_ion.m, single_ion.T0, flr.gamma0_pade)
        pol_op_pade_avg_ion[i,j] = ctx.polarization_op_single_ion(kperp, ctx.ntot_ion, ctx.qavg_ion, ctx.mavg_ion, ctx.Tavg_ion, flr.gamma0_pade)

fig2, axs = plt.subplots(3,2, figsize=(8,9))
axs = axs.flatten()

ax = axs[0]
Z = (pol_op_ex - pol_op_pade)/np.abs(pol_op_ex)*100
im = ax.pcolormesh(qratios, nratios, Z, shading='auto',cmap='bwr', vmin=vmin_pade, vmax=vmax_pade)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'Charge number $Z_{W}$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'Exact - Pade ($k_\perp \rho_D=%.1f$)'%kperprhoD)

ax = axs[1]
Z = (pol_op_ex - pol_op_pade)/(2*pol_op_ex - pol_op_pade)*100
im = ax.pcolormesh(qratios, nratios, Z, shading='auto',cmap='bwr', vmin=vmin_pade, vmax=vmax_pade)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'Charge number $Z_{W}$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'$\delta\phi/\phi$ ($k_\perp \rho_D=%.1f$)'%kperprhoD)

ax = axs[2]
Z = (pol_op_ex - pol_op_pade_single_ion)/np.abs(pol_op_ex)*100
im = ax.pcolormesh(qratios, nratios, Z, shading='auto',cmap='bwr', vmin=vmin_single, vmax=vmax_single)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'Charge number $Z_{W}$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'Exact - Pade single ion ($k_\perp \rho_D=%.1f$)'%kperprhoD)

ax = axs[3]
Z = (pol_op_ex - pol_op_pade_single_ion)/(2*pol_op_ex - pol_op_pade_single_ion)*100
im = ax.pcolormesh(qratios, nratios, Z, shading='auto',cmap='bwr', vmin=vmin_single, vmax=vmax_single)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'Charge number $Z_{W}$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'$\delta\phi/\phi$ ($k_\perp \rho_D=%.1f$)'%kperprhoD)

ax = axs[4]
Z = (pol_op_ex - pol_op_pade_avg_ion)/np.abs(pol_op_ex)*100
im = ax.pcolormesh(qratios, nratios, Z, shading='auto',cmap='bwr', vmin=vmin_avg, vmax=vmax_avg)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'Charge number $Z_{W}$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'Exact - Pade avg ion ($k_\perp \rho_D=%.1f$)'%kperprhoD) 

ax = axs[5]
Z = (pol_op_ex - pol_op_pade_avg_ion)/(2*pol_op_ex - pol_op_pade_avg_ion)*100
im = ax.pcolormesh(qratios, nratios, Z, shading='auto',cmap='bwr', vmin=vmin_phi, vmax=vmax_phi)
cbar = fig2.colorbar(im, ax=ax, label=r'\%')
ax.set_xlabel(r'Charge number $Z_{W}$')
ax.set_ylabel(r'$n_{Wn+}/n_D$')
ax.set_yscale('log')
ax.set_title(r'$\delta\phi/\phi$ ($k_\perp \rho_D=%.1f$)'%kperprhoD)

plt.tight_layout()
plt.show()